In [1]:
import os

In [2]:
os.chdir('../')
%pwd

'd:\\DHIRAJ\\Data_Science\\Jupyter_Workspace\\Projects\\Text-Summarization'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int


In [4]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.TrainingArguments
        create_directories([config.root_dir])

        data_model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt=config.model_ckpt,
            num_train_epochs=params.num_train_epochs,
            warmup_steps=params.warmup_steps,
            per_device_train_batch_size=params.per_device_train_batch_size,
            per_device_eval_batch_size=params.per_device_eval_batch_size,
            weight_decay=params.weight_decay,
            logging_steps=params.logging_steps,
            evaluation_strategy=params.evaluation_strategy,
            eval_steps=params.eval_steps,
            save_steps=params.save_steps,
            gradient_accumulation_steps=params.gradient_accumulation_steps
            )
        
        return data_model_trainer_config


In [6]:
from transformers import AutoTokenizer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM
from transformers import Trainer, TrainingArguments
import torch
from datasets import load_from_disk, load_dataset


c:\Users\Dhiraj\anaconda3\envs\nlp_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-05-16 21:03:08,860: INFO: config: PyTorch version 2.7.0 available.]


In [7]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_pegasus)

        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(output_dir=self.config.root_dir,
                                         num_train_epochs=self.config.num_train_epochs,
                                         warmup_steps=self.config.warmup_steps,
                                         per_device_train_batch_size=self.config.per_device_train_batch_size,
                                         per_device_eval_batch_size=self.config.per_device_eval_batch_size,
                                         weight_decay=self.config.weight_decay,
                                         logging_steps=self.config.logging_steps,
                                         evaluation_strategy=self.config.evaluation_strategy,
                                         eval_steps=self.config.eval_steps,
                                         save_steps=self.config.save_steps,
                                         gradient_accumulation_steps=self.config.gradient_accumulation_steps
                                        )
        
        trainer = Trainer(model=model_pegasus,
                          args=trainer_args,
                          tokenizer=tokenizer,
                          data_collator=seq2seq_data_collator,
                          train_dataset=dataset_samsum_pt["test"],      # change to train
                          eval_dataset=dataset_samsum_pt['validation']
                          )
        
        trainer.train()

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, 'pegasus-samsum-model'))

        tokenizer.save_pretrained(os.path.join(self.config.root_dir, 'tokenizer'))

In [8]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2025-05-16 21:03:15,147: INFO: common: yaml file : config\config.yaml loaded successfully]
[2025-05-16 21:03:15,160: INFO: common: yaml file : params.yaml loaded successfully]
[2025-05-16 21:03:15,173: INFO: common: created directory at : artifacts]
[2025-05-16 21:03:15,178: INFO: common: created directory at : artifacts/model_trainer]


OSError: google/pegasus-cnn_dailymail does not appear to have a file named pytorch_model.bin, model.safetensors, tf_model.h5, model.ckpt or flax_model.msgpack.